In [141]:
import pandas as pd
import os

In [121]:
def descarga_csv(dataframe, nombre_carpeta, nombre_archivo):
    """
    Guarda el DataFrame en un archivo CSV en la carpeta destino.

    Args:
        dataframe (pd.DataFrame): El DataFrame a guardar.
        nombre_carpeta (str): La ruta de la carpeta donde se guarda el archivo.
        nombre_archivo (str): El nombre del archivo CSV.
    """
    try:
        if not os.path.exists(nombre_carpeta):
            os.makedirs(nombre_carpeta)
            print(f"Se creó la carpeta: '{nombre_carpeta}'")

        ruta_completa = os.path.join(nombre_carpeta, nombre_archivo)
        dataframe.to_csv(ruta_completa, index=False, encoding='utf-8')
        
        print(f"Archivo con dataframe de educacion y salud creado en: '{ruta_completa}'")

    except Exception as e:
        print(f"Ocurrió un error al guardar el archivo: {e}")

In [123]:
# Carga de datos raw
df_raw = pd.read_csv('../data/raw/educacionysalud.csv') 

In [125]:
#Cambio de formato long a wide usando pivot_table
df_raw['valor'] = pd.to_numeric(df_raw['valor'], errors='coerce')
df_wide = df_raw.pivot_table(index=['id_estado','estado', 'año'], columns = 'indicador_nombre', values = 'valor').reset_index()
df_wide.columns = ['id_estado','estado', 'fecha', 'derhab', 'pob_bac', 'pob_edbas', 'pob_edsup', 'porc_analf', 'porc_sined', 'promedio_ed']

In [127]:
# Diagnóstico rápido
print("--- Dimensiones del DataFrame ---")
print(df_wide.shape)

print("\n--- Primeras filas ---")
print(df_wide.head())

print("\n--- Tipos de datos y nulos ---")
print(df_wide.info())

print("\n--- Estadísticas descriptivas ---")
print(df_wide.describe())

--- Dimensiones del DataFrame ---
(479, 10)

--- Primeras filas ---
   id_estado          estado  fecha  derhab  pob_bac  pob_edbas  pob_edsup  \
0          1  Aguascalientes   1910     NaN      NaN        NaN        NaN   
1          1  Aguascalientes   1921     NaN      NaN        NaN        NaN   
2          1  Aguascalientes   1930     NaN      NaN        NaN        NaN   
3          1  Aguascalientes   1940     NaN      NaN        NaN        NaN   
4          1  Aguascalientes   1950     NaN      NaN        NaN        NaN   

   porc_analf  porc_sined  promedio_ed  
0        64.3         NaN          NaN  
1        53.8         NaN          NaN  
2        56.1         NaN          NaN  
3        39.8         NaN          NaN  
4        31.0         NaN          NaN  

--- Tipos de datos y nulos ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  ----

In [129]:
df_wide

,id_estado,estado,fecha,derhab,pob_bac,pob_edbas,pob_edsup,porc_analf,porc_sined,promedio_ed
0,1,Aguascalientes,1910,NaN,NaN,NaN,NaN,64.300000,NaN,NaN
1,1,Aguascalientes,1921,NaN,NaN,NaN,NaN,53.800000,NaN,NaN
2,1,Aguascalientes,1930,NaN,NaN,NaN,NaN,56.100000,NaN,NaN
3,1,Aguascalientes,1940,NaN,NaN,NaN,NaN,39.800000,NaN,NaN
4,1,Aguascalientes,1950,NaN,NaN,NaN,NaN,31.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
474,32,Zacatecas,2000,438329.0,43715.0,617071.0,49138.0,8.000000,NaN,6.500000
475,32,Zacatecas,2005,577422.0,NaN,NaN,NaN,7.200000,NaN,7.200000
476,32,Zacatecas,2010,1020487.0,89648.0,687233.0,98050.0,5.550000,NaN,7.900000
477,32,Zacatecas,2015,NaN,NaN,NaN,NaN,4.387647,4.924372,8.629599


In [143]:
#Creacion de copia para limpieza de datos
df_interim = df_wide.copy()

#LIMPIEZA DE DATOS

#Tipo de dato
df_interim['estado'] = df_interim['estado'].astype('category')
df_interim['id_estado'] = df_interim['id_estado'].astype('category')

#Filtramos por fecha, solo se utilizaran datos del 2000 en adelante
df_interim = df_interim[df_interim['fecha'] >= 2000].copy()

#Guardado de datos /interim/
descarga_csv(df_interim, nombre_carpeta='../data/interim/', nombre_archivo='educacionysalud_2000_filtrado.csv')

#Datos nulos
# Calcula el porcentaje de nulos por columna, si es mayor a 60, eliminaremos la columna
porcentaje_nulos = df_interim.isnull().sum() / len(df_interim) * 100
umbral = 60
drop_columnas = porcentaje_nulos[porcentaje_nulos >= umbral].index.tolist()
df_limpio = df_interim.drop(columns= drop_columnas)


#Imputación con interpolación lineal
#Ordena los datos por estado y año
df_limpio = df_limpio.sort_values(by=['estado', 'fecha'])

#Creación de columna indicadora de imputación
columnas_a_imputar = ['derhab', 'porc_analf', 'promedio_ed']
for col in columnas_a_imputar:
    # Solo crea la columna si existe en df_limpio
    if col in df_limpio.columns:
        nombre_col_indicador = f'{col}_imputado'
        df_limpio[nombre_col_indicador] = df_limpio[col].isnull().astype(int)

# Agrupa por estado y aplica la interpolación a cada grupo
df_limpio[columnas_a_imputar] = df_limpio.groupby('estado', observed=False)[columnas_a_imputar].transform(
    lambda x: x.interpolate(method='linear', limit_direction='both')
)

print("Datos nulos después de la interpolación:")
print(df_limpio.isnull().sum())

Archivo con dataframe de educacion y salud creado en: '../data/interim/educacionysalud_2000_filtrado.csv'
Datos nulos después de la interpolación:
id_estado               0
estado                  0
fecha                   0
derhab                  0
porc_analf              0
promedio_ed             0
derhab_imputado         0
porc_analf_imputado     0
promedio_ed_imputado    0
dtype: int64


In [145]:
#PRUEBAS DE CALIDAD
# Regla 1: No debe haber valores nulos en columnas clave.
assert df_limpio['estado'].isnull().sum() == 0, "La columna 'estado' no puede tener nulos."
assert df_limpio['fecha'].isnull().sum() == 0, "La columna 'fecha' no puede tener nulos."

# Regla 2: Los valores de los indicadores deben estar en un rango lógico.
# (Ej. los porcentajes deben estar entre 0 y 100)
assert df_limpio['porc_analf'].between(0, 100).all(), "Hay porcentajes de analfabetismo fuera del rango 0-100."

# Regla 3: Consistencia categórica de estados, el número de estados únicos debe ser 32
assert df_limpio['estado'].nunique() == 32, f"Se esperaban 32 estados únicos, pero se encontraron {df_a_validar['estado'].nunique()}."
assert df_limpio['id_estado'].nunique() == 32, f"Se esperaban 32 estados únicos, pero se encontraron {df_a_validar['estado'].nunique()}."

# Regla 4: Unicidad de registros.
# (Ej. no debe haber filas duplicadas para la misma combinación de estado y año)
assert not df_limpio.duplicated(subset=['estado', 'fecha']).any(), "Existen registros duplicados de estado-fecha."

print("Todas las reglas de calidad han pasado exitosamente.")

# Guardado final de datos usando la función que creamos
descarga_csv(df_limpio, nombre_carpeta='../data/processed/', nombre_archivo='educacionysalud_procesado.csv')

Todas las reglas de calidad han pasado exitosamente.
Archivo con dataframe de educacion y salud creado en: '../data/processed/educacionysalud_procesado.csv'
